In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout, Output, Label
from IPython.display import display, Latex, Image, Markdown

import params as st
from model import linSys, nonlinSys
from feedforward import *

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

# Steuerungen beim Brückenkran

In [ ]:
imag = Image("../images/kran.png", width=300)

outL = Output()
outR = Output()

with outL:
    display(imag)
with outR:
    display(Markdown("""
**Modellgleichungen**

nichtlinear |   | linear
----------------------|----|------------------------
\\begin{align*}
    M \\ddot{D}(t) & = u(t) + F(t) \\langle\\mathbf{\\eta}(t),\\mathbf{e}_2 \\rangle \\\\
    m (\\ddot{\\mathbf{y}}(t) - \\mathbf{g}) & = -F(t) \\mathbf{\\eta}(t) \\\\
    \\mathbf{y}(t) &  = D(t) \\mathbf{e}_2 + l \\mathbf{\\eta}(t)
\\end{align*} |   | \\begin{align*}
    M \\ddot{D}(t) & = u(t) + m g \\theta(t) \\\\
    m \\ddot{y}(t) & = -m g \\theta(t) \\\\
    y(t) &  = D(t) + l \\theta(t)
\\end{align*}

**flachheitsbasierte Steuerung auf Basis des linearen Modells**

Entwurf |   | Eingang
----------------------|----|------------------------
\\begin{align*}
    y_{\\mathrm{r}}(t) & = \\begin{cases}
    y_0, & t \\le t_0 \\\\
    y_0 + y_{\\mathrm{d}} \\varphi_4(t/T) & t_0 < t < t_0 + T \\\\
    y_{\\mathrm{d}} & t \\ge t_0 + T
    \\end{cases}
\\end{align*} |   | \\begin{align*}
    u_{\\mathrm{r}}(t) & = \\frac{M l}{g}  y_{\\mathrm{r}}^{(4)}(t) + (M + m)  \\ddot{y}_{\\mathrm{r}}(t)\\\\
    D_{\\mathrm{r}}(t) & =  y_{\\mathrm{r}}(t) + \\frac{l}{g} \\ddot{y}_{\\mathrm{r}}(t) \\\\
    \\theta_{\\mathrm{r}}(t) & = - \\frac{ \\ddot{y}_{\\mathrm{r}}(t)}{g}
\\end{align*}

    """))
cols = HBox([outL, outR], layout=Layout(display='flex', flex_flow='row', justify_content='space-around', align_items='center'))
display(cols)

**Definition Parameter**

In [ ]:
tSim = np.linspace(0, 10, 1001)

In [ ]:
output = widgets.Output()

with output:
    fig = plt.figure(figsize=(10, 5))
    ax3 = plt.subplot(222)
    ax4 = plt.subplot(224)
    ax1 = plt.subplot(221)
    ax2 = plt.subplot(223)

plt.subplots_adjust(wspace=0.2, hspace=0.3)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1.set_xlim([tSim[0], tSim[-1]]) 
ax2.set_xlim([tSim[0], tSim[-1]]) 
ax4.set_xlim([tSim[0], tSim[-1]]) 
ax1.set_ylim([-15, 15]) 
ax2.set_ylim([-0.1, 1.5]) 
ax4.set_ylim([-20, 20]) 
ax1.grid() 
ax2.grid()
ax4.grid()
ax3.set_xlim(-0.3, 1.3)
ax3.set_ylim(-st.l - 2 * st.r, 2 * st.h)
ax3.set_xticks([])
ax3.set_yticks([])
ax1.set_ylabel(r"$u$")
ax2.set_xlabel(r"$t$")
ax2.set_ylabel(r"$y$ in m")
ax4.set_ylabel(r"$\theta$ in °")
ax4.set_xlabel(r"$t$")

lineY, = ax2.plot([], [], label=r"Istverlauf")
lineYref, = ax2.plot([], [], '--', label=r"Sollverlauf")
lineTheta, = ax4.plot([], [])
lineThetaref, = ax4.plot([], [], '--')
lineU, = ax1.plot([], [])

pathAni = ax3.plot([-0.3, 1.3], [0, 0], zorder=0, color='C6')
seilAni, = ax3.plot([0, 0], [0, -st.l], zorder=0, color='0')
lastAni = ax3.add_patch(plt.Circle((0,-st.l), st.r, color='0' , zorder=1))
wagenAni = ax3.add_patch(plt.Rectangle((-st.b / 2, 0), st.b, st.h, facecolor='0.', edgecolor='0.'))

handlesAx, labelsAx = ax2.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.94, 0.7735, .15), loc=3,
           ncol=2, mode="expand", borderaxespad=0., framealpha=0.5)

playB = widgets.Play(value=0,
                     min=0, 
                     max=len(tSim),
                     step=10)
sliderB = widgets.IntSlider(value=0,
                            min=0,
                            max=len(tSim),
                            step=10)
sliderT0 = widgets.FloatSlider(value=0,
                               min=0,
                               max=3,
                               step=1,
                               description=r'$t_0$')
sliderT = widgets.FloatSlider(value=1.6,
                               min=1.5,
                               max=2,
                               step=.1,
                               description=r'$T$')
sliderYd = widgets.widgets.FloatSlider(value=1,
                                       min=0.7,
                                       max=1.2,
                                       step=0.1,
                                       description=r"$y_\text{d}$")
radioU = widgets.RadioButtons(options=['Sprung', 'Steuerung: flach linear', 'Filter'],
                              disabled=False)
radioMod = widgets.RadioButtons(options=['linear', 'nichtlinear'],
                              disabled=False)

def updateOde(_):
    global res

    t0 = sliderT0.value
    T = sliderT.value
    yd = sliderYd.value
    sType = radioMod.value

    def sys(t, x, u, sType, params):
        if sType == 'linear':
            dx = linSys(t, x, u, params)
        else:
            dx = nonlinSys(t, x, u, params)

        return dx

    params = st.g, st.M, st.m , st.l
    x0 = [0, 0, 0, 0]
    if radioU.value == 'Sprung':
        uIn = feedForwardTrapz(t0, T, yd)
    elif radioU.value == 'Filter':
        _, uIn = feedForwardFilter(t0, yd)
    else:
        _, uIn, _, _ = feedForwardLinFlat(t0, T, yd)
    
    res = solve_ivp(sys,
                    [tSim[0], tSim[-1]],
                    x0,
                    t_eval=tSim,
                    args=(uIn, sType, params))

def updatePlot(change):
    idx = change['new']

    theta = res.y.T[idx, 2]
    D = res.y.T[idx, 0]
    y1 = D + np.sin(theta) * st.l 
    y2 = np.cos(theta) * st.l
    
    t0 = sliderT0.value
    T = sliderT.value
    yd = sliderYd.value
    if radioU.value == 'Sprung':
        uIn = feedForwardTrapz(t0, T, yd)
        lineThetaref.set_data([], [])
        lineYref.set_data([], [])
        thrMin = np.min(np.rad2deg(res.y.T[:, 2]))
        thrMax = np.max(np.rad2deg(res.y.T[:, 2]))
        yrMin = np.min(res.y.T[:, 0] + np.sin(res.y.T[:, 2]) * st.l)
        yrMax = np.max(res.y.T[:, 0] + np.sin(res.y.T[:, 2]) * st.l)
    elif radioU.value == 'Filter':
        yr, uIn = feedForwardFilter(t0, yd)
        lineThetaref.set_data([], [])
        lineYref.set_data(tSim[:idx], [yr[0](_t) for _t in tSim[:idx]])
        thrMin = np.min(np.rad2deg(res.y.T[:, 2]))
        thrMax = np.max(np.rad2deg(res.y.T[:, 2]))
        yrMin = np.min([yr[0](_t) for _t in tSim])
        yrMax = np.max([yr[0](_t) for _t in tSim])
    else:
        yr, uIn, Dr, thetar = feedForwardLinFlat(t0, T, yd)
        lineThetaref.set_data(tSim[:idx], [np.rad2deg(thetar(_t)) for _t in tSim[:idx]])
        lineYref.set_data(tSim[:idx], [yr[0](_t) for _t in tSim[:idx]])
        thrMin = np.min([np.rad2deg(thetar(_t)) for _t in tSim])
        thrMax = np.max([np.rad2deg(thetar(_t)) for _t in tSim])
        yrMin = np.min([yr[0](_t) for _t in tSim])
        yrMax = np.max([yr[0](_t) for _t in tSim])

    lineU.set_data(tSim[:idx], [uIn(_t) for _t in tSim[:idx]])
    lineTheta.set_data(tSim[:idx], np.rad2deg(res.y.T[:idx, 2]))
    lineY.set_data(tSim[:idx], res.y.T[:idx, 0] + np.sin(res.y.T[:idx, 2]) * st.l)

    uMin = np.min([uIn(_t) for _t in tSim])
    uMax = np.max([uIn(_t) for _t in tSim])
    thMin = np.minimum(np.min(np.rad2deg(res.y.T[:, 2])), thrMin)
    thMax = np.maximum(np.max(np.rad2deg(res.y.T[:, 2])), thrMax)    
    yMin = np.minimum(np.min(res.y.T[:, 0] + np.sin(res.y.T[:, 2]) * st.l), yrMin)
    yMax = np.maximum(np.max(res.y.T[:, 0] + np.sin(res.y.T[:, 2]) * st.l), yrMax)
    
    ax1.set_ylim(uMin - np.abs(uMax - uMin) * 0.1, uMax + np.abs(uMax - uMin) * 0.1)
    ax2.set_ylim(yMin - np.abs(yMax - yMin) * 0.1, yMax + np.abs(yMax - yMin) * 0.1)
    ax4.set_ylim(thMin - np.abs(thMax - thMin) * 0.1, thMax + np.abs(thMax - thMin) * 0.1)

    seilAni.set_data([D, y1] ,[0, -y2])
    lastAni.set_center((y1, -y2))
    wagenAni.set_x(D - st.b/2)
    
    fig.canvas.draw()    

sliderB.observe(updatePlot, names='value')
sliderT0.observe(updateOde, names='value')
sliderT.observe(updateOde, names='value')
sliderYd.observe(updateOde, names='value')
radioU.observe(updateOde, names='value')
radioMod.observe(updateOde, names='value')

updateOde(_)

bFeedForward = VBox([Label(value='Steuerung'), sliderYd, sliderT0, sliderT])
bFeedForward.layout = make_box_layout()
bInput = VBox([Label(value='Eingang'), radioU])
bInput.layout = make_box_layout()
bModel = VBox([Label(value='Modell'), radioMod])
bModel.layout = make_box_layout()
controls = HBox([bFeedForward, bInput, bModel])

jslink((playB, 'value'), (sliderB, 'value'))
videoControls = VBox([HBox([playB, sliderB]), output])
videoControls.layout = make_box_layout()

HBox([controls, Box([videoControls])], layout=Layout(display='flex', flex_flow='column', justify_content='center', align_items='center'))

In [ ]:
res.y.T

In [ ]:
t0 = sliderT0.value
T = sliderT.value
yd = sliderYd.value
sType = radioMod.value

def sys(t, x, u, sType, params):
    if sType == 'linear':
        dx = linSys(t, x, u, params)
    else:
        dx = nonlinSys(t, x, u, params)

    return dx

params = st.g, st.M, st.m , st.l
x0 = [0, 0, 0, 0]
if radioU.value == 'Sprung':
    uIn = feedForwardTrapz(t0, T, yd)
elif radioU.value == 'Filter':
    _, uIn = feedForwardFilter(t0, yd)
else:
    _, uIn, _, _ = feedForwardLinFlat(t0, T, yd)

res = solve_ivp(sys,
                [tSim[0], tSim[-1]],
                x0,
                t_eval=tSim,
                args=(uIn, sType, params))

In [ ]:
res.y

In [ ]:
res.y[:,-1]